In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.5 MB/s eta 0:00:00


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from google.colab import drive

In [3]:
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
#  Import and read the diabetes_binary_5050.csv.
diabetes_df = pd.read_csv("/content/drive/MyDrive/Dataset/diabetes_data_updated.csv")
diabetes_df.head()

,Diabetes_binary,Age,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Education,Income,Unique_ID
0,0.0,7,1,0.0,1,33.0,0.0,0.0,0.0,1,...,1,0.0,2.0,15.0,0.0,1.0,1,6.0,9.0,18861
1,0.0,7,0,1.0,1,27.0,1.0,0.0,0.0,1,...,1,0.0,2.0,1.0,2.0,0.0,1,6.0,6.0,20160
2,0.0,13,0,1.0,1,26.0,1.0,0.0,0.0,0,...,1,0.0,3.0,0.0,30.0,0.0,1,4.0,3.0,22983
3,0.0,11,0,0.0,1,19.0,1.0,0.0,0.0,1,...,1,0.0,3.0,0.0,0.0,0.0,0,5.0,7.0,19244
4,0.0,5,1,0.0,1,37.0,0.0,0.0,0.0,1,...,1,0.0,2.0,0.0,0.0,0.0,0,5.0,3.0,29881


In [5]:
# Determine the number of unique values in each column.
diabetes_df.nunique()

Diabetes_binary             2
Age                        13
HighBP                      2
HighChol                    2
CholCheck                   2
BMI                        79
Smoker                      2
Stroke                      2
HeartDiseaseorAttack        2
PhysActivity                2
Fruits                      2
Veggies                     2
HvyAlcoholConsump           2
AnyHealthcare               2
NoDocbcCost                 2
GenHlth                     5
MentHlth                   31
PhysHlth                   31
DiffWalk                    2
Sex                         2
Education                   6
Income                     11
Unique_ID               67136
dtype: int64

In [6]:
# Drop the non-beneficial columns
diabetes_df = diabetes_df.drop(columns = ["Veggies", "Fruits", "Unique_ID"], axis = 1)
diabetes_df.head()

,Diabetes_binary,Age,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Education,Income
0,0.0,7,1,0.0,1,33.0,0.0,0.0,0.0,1,0,1,0.0,2.0,15.0,0.0,1.0,1,6.0,9.0
1,0.0,7,0,1.0,1,27.0,1.0,0.0,0.0,1,0,1,0.0,2.0,1.0,2.0,0.0,1,6.0,6.0
2,0.0,13,0,1.0,1,26.0,1.0,0.0,0.0,0,0,1,0.0,3.0,0.0,30.0,0.0,1,4.0,3.0
3,0.0,11,0,0.0,1,19.0,1.0,0.0,0.0,1,0,1,0.0,3.0,0.0,0.0,0.0,0,5.0,7.0
4,0.0,5,1,0.0,1,37.0,0.0,0.0,0.0,1,0,1,0.0,2.0,0.0,0.0,0.0,0,5.0,3.0


In [7]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(diabetes_df)
dummies_df.head()

,Diabetes_binary,Age,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Education,Income
0,0.0,7,1,0.0,1,33.0,0.0,0.0,0.0,1,0,1,0.0,2.0,15.0,0.0,1.0,1,6.0,9.0
1,0.0,7,0,1.0,1,27.0,1.0,0.0,0.0,1,0,1,0.0,2.0,1.0,2.0,0.0,1,6.0,6.0
2,0.0,13,0,1.0,1,26.0,1.0,0.0,0.0,0,0,1,0.0,3.0,0.0,30.0,0.0,1,4.0,3.0
3,0.0,11,0,0.0,1,19.0,1.0,0.0,0.0,1,0,1,0.0,3.0,0.0,0.0,0.0,0,5.0,7.0
4,0.0,5,1,0.0,1,37.0,0.0,0.0,0.0,1,0,1,0.0,2.0,0.0,0.0,0.0,0,5.0,3.0


In [8]:
# Split our preprocessed data into our features and target arrays
y = dummies_df["Diabetes_binary"].values
X = dummies_df.drop(["Diabetes_binary"], axis = 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, stratify = y)

In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Create a method that creates a new Sequential model with hyperparameter options (hp)
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to test in hidden layers
    activation = hp.Choice("activation", ["relu", "tanh", "sigmoid"])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units = hp.Int("first_units",
        min_value = 1,
        max_value = 10,
        step = 2), activation = activation, input_dim = len(X_train[0])))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int("num_layers", 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units = hp.Int("units_" + str(i),
            min_value = 1,
            max_value = 10,
            step = 2),
            activation = activation))

    nn_model.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

    # Compile the model
    nn_model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

    return nn_model

In [11]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective = "val_accuracy",
    max_epochs = 20,
    hyperband_iterations = 2)

In [12]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train, epochs = 20, validation_data = (X_test_scaled, y_test))

Trial 60 Complete [00h 01m 50s]
val_accuracy: 0.7504766583442688

Best val_accuracy So Far: 0.7504766583442688
Total elapsed time: 00h 54m 18s


In [13]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 7,
 'num_layers': 1,
 'units_0': 5,
 'units_1': 1,
 'units_2': 9,
 'units_3': 7,
 'units_4': 5,
 'units_5': 9,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [14]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test,verbose = 2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

525/525 - 1s - loss: 0.5102 - accuracy: 0.7505 - 935ms/epoch - 2ms/step
Loss: 0.5102412104606628, Accuracy: 0.7504766583442688
525/525 - 1s - loss: 0.5107 - accuracy: 0.7492 - 917ms/epoch - 2ms/step
Loss: 0.5106855034828186, Accuracy: 0.7492254376411438
525/525 - 1s - loss: 0.5132 - accuracy: 0.7491 - 1s/epoch - 3ms/step
Loss: 0.5132323503494263, Accuracy: 0.7491062879562378


In [15]:
# Get second best model hyperparameters
second_hyper = tuner.get_best_hyperparameters(2)[1]
second_hyper.values

{'activation': 'relu',
 'first_units': 9,
 'num_layers': 2,
 'units_0': 7,
 'units_1': 7,
 'units_2': 7,
 'units_3': 3,
 'units_4': 3,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0021'}